In [1]:
import gc
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
X_test_1 = pd.read_feather('../input/amex-imputed-and-1hot-encoded/X_test_1.ftr').set_index('customer_ID')
X_test_2 = pd.read_feather('../input/amex-imputed-and-1hot-encoded/X_test_2.ftr').set_index('customer_ID')

In [3]:
model = tf.keras.models.load_model('/kaggle/input/test-dtc-model/shallow_nn.h5')

2023-02-11 02:28:30.805924: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [4]:
preds1 = pd.DataFrame(model.predict(dict(X_test_1), batch_size=5000, verbose=1), index=X_test_1.index, columns=['prediction'])
preds2 = pd.DataFrame(model.predict(dict(X_test_2), batch_size=5000, verbose=1), index=X_test_2.index, columns=['prediction'])

2023-02-11 02:28:37.380654: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


1137/1137 [==============================] - 43s 37ms/step


In [5]:
del X_test_1, X_test_2
gc.collect()

11178

In [6]:
submission = pd.concat([preds1, preds2])

# predictions only need to be for each customer
submission = submission.groupby('customer_ID').agg(['last'])
submission.columns = submission.columns.droplevel(1)

# index needs to be removed from submission csv
submission = submission.reset_index()
submission.to_csv('submission.csv', index=False)